In [ ]:
## Assume data is in the processing container for this notebook
#pip install pymongo

In [42]:
import azureml.core
import cv2
import os, uuid, sys
import random
import tempfile
import json
import pymongo
import re
from azureml.core import Workspace, Datastore, Dataset
from azure.core._match_conditions import MatchConditions
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.filedatalake._models import ContentSettings
from datetime import datetime



def initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id):
    
    try:  
        global service_client
        global processing_file_system_client
        global processed_dir_system_client

        credential = ClientSecretCredential(tenant_id, client_id, client_secret)

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=credential)
    
        processing_file_system_client = service_client.get_file_system_client(file_system="processing")
        
        processed_dir_system_client = service_client.get_file_system_client(file_system="processed")
        
    except Exception as e:
        print(e)
        

def Rand(start, end, num):
    return random.sample(range(start, end), num)

def create_directory(container, directory):
    folders = directory.split('/')
    #folders.pop(0) #remove dropZone
    #folders.pop(0) #remove ClientVideos
    print(folders)
    try:
        file_system_client = service_client.get_file_system_client(file_system=container)
        dir = ''
        for f in folders:
            try:
                dir = dir + '/' + f
                #print(dir)
                file_system_client.create_directory(dir)
            except Exception as e:
                print(e)
                pass
                
    except Exception as e:
        print(e)
            

def mount_mp4(file_fullpath, file_path, file_name, path):
    print(file_fullpath)
    dataset = Dataset.File.from_files((datastore, file_fullpath))
    mounted_path = tempfile.mkdtemp()
    mount_context = dataset.mount(mounted_path)
    mount_context.start()
    print('list directory')
    print(os.listdir(mounted_path))
    print (mounted_path)
    vid_cap = cv2.VideoCapture(mounted_path + '/' + file_name)
    if not vid_cap.isOpened():
        print('Cannot open file')

    frame_count = 0

    while(vid_cap.isOpened()):

        totalframecount= int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))        
        print("The total number of frames in this video is ", totalframecount)
        vals = Rand(0, totalframecount, 10)
        print(vals)
        
        for i in vals:
            frame_count = i
            vid_cap.set(1, frame_count)
            _, frame = vid_cap.read()
            # Show frame size
            h, w = frame.shape[:2]
            print("\n\n Frame height = {} \n Frame width = {} \n\n ".format(h,w))

            # create naming structure
            now = datetime.now()  
            filetime = now.strftime("%Y%d%m%H%M%S%f")
            frame_num = str(frame_count).zfill(6)
            frame_file_name = f"Frame{frame_num}-{filetime}-raw.jpg"
            frame_encoded = cv2.imencode(".jpg", frame)[1].tobytes()
        
            try:
                directory = os.path.dirname(file_fullpath)
                print('directory = ' + directory)
                create_directory('processed', directory)    
                
                file_system_client = service_client.get_file_system_client(file_system='processed')
                directory_client = file_system_client.get_directory_client(directory)
                
                file_client = directory_client.create_file(frame_file_name)
                file_client.append_data(data=frame_encoded, offset=0, length=len(frame_encoded))
                file_client.flush_data(len(frame_encoded))
                json_obj = jsonifypath(path)
                
                print(f"Uploaded image: {frame_file_name}")

            except Exception as e:
                print(e)
        vid_cap.release() 
    mount_context.stop()

    
def process_contents():
    try:
        processing_paths = processing_file_system_client.get_paths(path="dropZone/clientVideo")           
        for path in processing_paths:
            if path.is_directory == True:
                processing_directory_client = service_client.get_directory_client(processing_file_system_client.file_system_name,path.name)
            else:
                folder_path = os.path.dirname(path.name)
                print('folder path = ' + folder_path) 
                file_name = os.path.basename(path.name)
                print('file name = ' + file_name)
                #mount_mp4(path.name, folder_path, file_name, path)
                jsonifypath(path)

    except Exception as e:
        print(e)

#datetime.datetime(2022, 4, 21, 21, 41, 52)
def remote_non_utf8(datetime):
     return re.sub(r'[^\x00-\x7f]',r' ',name)
    
def jsonifypath(path):
    strpath = str(path)
    strpath = strpath.replace("'is_directory': False", "'is_directory': false")
    strpath = strpath.replace("'is_directory': True", "'is_directory': true")
    strpath = strpath.replace("datetime.datetime", "")
    strpath = strpath.replace("(", "'")
    strpath = strpath.replace(")", "'")
    strpath = strpath.replace('\'', '"')
    json_data = json.loads(strpath)
    json_data["camera"] = "camera_001"
    print(json_data)
    return strpath
    
    
def insert_mongo(data):
    database_name = 'imagedb'
    collection_name = 'image'
    uri = "getfromportal"
    client = pymongo.MongoClient(uri)
    database = client[database_name]
    container = database[collection_name]
    container.insert_one(data)

In [43]:
ws = Workspace.from_config()
datastore = Datastore.get(ws, datastore_name='adls_processing')
storage_account_name = 'storageaccountname'
client_id = 'XXX-XXX-XXX-XXX'
client_secret = 'topsecret'
tenant_id = 'XXX-XXX-XX-XXX'
initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id)
process_contents()

folder path = dropZone/clientVideo/client_001/04-21-2022
file name = AppleAndBanana.mp4
{'name': 'dropZone/clientVideo/client_001/04-21-2022/AppleAndBanana.mp4', 'owner': '$superuser', 'group': '$superuser', 'permissions': 'rw-r-----', 'last_modified': '2022, 4, 21, 21, 41, 52', 'is_directory': False, 'etag': '0x8DA23DFBF8B1534', 'content_length': 12034147, 'camera': 'camera_001'}
